# Serverless Example
## Tensorflow 2.0 [ CLASSIFICATION ]

## Setup

In [0]:
# load some dependencies
!pip install tensorflow==2.0.0-beta1
!pip install -q tf-nightly-2.0-preview

import os
import time
import logging
import numpy as np
import pandas as pd
import tensorflow as tf
from collections import OrderedDict
from sklearn import datasets
from sklearn.model_selection import train_test_split

tf.__version__

'2.0.0-dev20190708'

### Load database

In [0]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

## Training Process

### Model Definition
We define a simple neural network. In TensorFlow.Keras we need to specify the input size using "Input" as first layer.

In [0]:
def my_model(input_size):
    tf.keras.backend.clear_session()

    # Create a custom model using tf.keras
    inputs = tf.keras.layers.Input(shape=(input_size))
    x = tf.keras.layers.Dense(8, activation='relu')(inputs)
    x = tf.keras.layers.Dense(4, activation='relu')(x)
    outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    return model

model = my_model(X.shape[1])

### Training
Time to train the model! The cross-validation is applied implicitly by the "fit" function.

In [0]:
def training(model, X, y):
    # Compile the model specifying optimizer, metrics and learning rate
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])    

    # A useful callback
    checkpoint = tf.keras.callbacks.ModelCheckpoint('model.hdf5', save_best_only=True)

    # Train the model
    model.fit(x=X,
              y=y,
              validation_split=0.2,
              batch_size=8,
              epochs=10,
              callbacks=[checkpoint],
              verbose=1)
    
    # Load the best model
    model = tf.keras.models.load_model('model.hdf5')
    return model
    

best_model = training(model, X, y)

Train on 120 samples, validate on 30 samples
Epoch 1/10
120/120 [==============================] - 0s 1ms/sample - loss: 2.3787 - accuracy: 0.4167 - val_loss: 2.1323 - val_accuracy: 0.0000e+00
Epoch 2/10
120/120 [==============================] - 0s 331us/sample - loss: 2.0144 - accuracy: 0.4167 - val_loss: 1.6324 - val_accuracy: 0.0000e+00
Epoch 3/10
120/120 [==============================] - 0s 353us/sample - loss: 1.7121 - accuracy: 0.4167 - val_loss: 1.2178 - val_accuracy: 0.0000e+00
Epoch 4/10
120/120 [==============================] - 0s 362us/sample - loss: 1.4677 - accuracy: 0.4167 - val_loss: 0.9829 - val_accuracy: 0.0000e+00
Epoch 5/10
120/120 [==============================] - 0s 346us/sample - loss: 1.2753 - accuracy: 0.4750 - val_loss: 0.8397 - val_accuracy: 0.2667
Epoch 6/10
120/120 [==============================] - 0s 367us/sample - loss: 1.1123 - accuracy: 0.5583 - val_loss: 0.7658 - val_accuracy: 0.8333
Epoch 7/10
120/120 [==============================] - 0s 228us/sa

## Testing Process
We simulate a test phase in which we only have a trained model saved in the current directory.

### Blank paper

In [0]:
# Lets put us in blank paper condition
del best_model

### Prediction
Load the model and simulate to predict the whole iris dataset.

In [0]:
logger = logging.getLogger('iris')


## Prediction 
def handle(event, **kwargs):
    # If data is received as json convert to pandas
    event = event['data'] if 'data' in event else event
    if not isinstance(event, pd.DataFrame):
        event = pd.DataFrame.from_dict(event, orient='columns')

    # Retrieve model from disk and use it for predictions
    model = tf.keras.models.load_model('model.hdf5')
    
    # Target format convertion
    target_dict = {0: 'setosa', 1: 'versicolor', 2:'virginica'}
    to_target = np.vectorize(lambda x: target_dict[x])
    
    return to_target(np.argmax(model(event.values), axis=1)).tolist()

## Testing and liveness check
def test(data, **kwargs):
    pred = handle(data)

    logger.warning(f"predicted: {pred}")
    
    return True


test(iris.data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
W0708 21:28:56.484899 140050027865984 <ipython-input-12-852f5552e1c1>:24] predicted: ['setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'setosa', '

True